In [1]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms, utils
import matplotlib.pyplot as plt

c:\Users\pavle\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = datasets.FashionMNIST(
    root = "fashionMNIST",
    train = True,
    download= True,
    transform = ToTensor(),
    target_transform = None
)
test_data = datasets.FashionMNIST(
    root = "fashionMNIST",
    train = False,
    download= True,
    transform = ToTensor(),
    target_transform = None
)


100%|██████████| 26421880/26421880 [00:03<00:00, 8485763.40it/s] 


Extracting fashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to fashionMNIST\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 867208.51it/s]


Extracting fashionMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to fashionMNIST\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 8087550.31it/s]


Extracting fashionMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to fashionMNIST\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 5150829.44it/s]

Extracting fashionMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to fashionMNIST\FashionMNIST\raw



In [3]:
from torch.utils.data import DataLoader
BS = 32
train_dataloader = DataLoader(dataset = train_data,
                              batch_size = BS,
                              shuffle = True)

test_dataloader = DataLoader(dataset = test_data,
                             batch_size = BS)
                              

In [4]:
class CNNClassifier(nn.Module):
  def __init__(self,
               input_shape,
               hidden_unit,
               output_shape):
    super().__init__()
    self.Conv_block_1  = nn.Sequential(
        nn.Conv2d(in_channels = input_shape,
                  out_channels = hidden_unit,
                  kernel_size = 2,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_unit,
                  out_channels = hidden_unit,
                  kernel_size = 2,
                  stride = 1,
                  padding = 1),

        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2))
    self.Conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels = hidden_unit,
                  out_channels = hidden_unit,
                  kernel_size= 2,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_unit,
                  out_channels = hidden_unit,
                  kernel_size= 2,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2)) ## output ce da bude matrix 10 8 8
    self.classifier_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_unit*8*8 ,####
                  out_features = output_shape))  
  
  def forward(self,x):
      x = self.Conv_block_1(x)
      x = self.Conv_block_2(x)
      x = self.classifier_layer(x)
      return x


In [5]:
model =  CNNClassifier(input_shape = 1,
                       hidden_unit = 10,
                       output_shape = 10)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [6]:
def train_step(model,
               data_loader,
               loss_fn,
               optimizer):
  model.train()

  train_acc, train_loss = 0,0

  for batch, (X,y) in enumerate(data_loader):

    model.train()

    y_pred = model(X)
    
    #loss and acc
    loss = loss_fn(y_pred, y)
    train_loss += loss

    acc = torch.argmax(y_pred, dim = 1)
    train_acc  += (acc == y).float().mean()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  train_acc /= len(data_loader)
  train_loss /= len(data_loader)

  print(f"train_loss = {train_loss:.2f}, train_acc = {train_acc:2f}")

In [7]:
def test_step(model,
              data_loader,
              loss_fn):
  test_loss, test_acc = 0,0

  model.eval()
  with torch.inference_mode():
    for X,y in data_loader:

      y_pred = model(X)

      loss =loss_fn(y_pred, y)
      test_loss += loss

      acc = torch.argmax(y_pred, dim = 1)
      test_acc  += (acc == y).float().mean()
      


    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"test_loss = {test_loss:2f} , test_acc = {test_acc:2f}")

In [8]:
from tqdm.auto import tqdm 

epochs = 5

for epoc in tqdm(range(epochs)):
  train_step(model,
             train_dataloader,
             loss_fn,
             optimizer)
  test_step(model,
            test_dataloader,
            loss_fn)

  0%|          | 0/5 [00:00<?, ?it/s]

train_loss = 0.58, train_acc = 0.792733


 20%|██        | 1/5 [00:28<01:53, 28.34s/it]

test_loss = 0.378862 , test_acc = 0.863918
train_loss = 0.35, train_acc = 0.873683


 40%|████      | 2/5 [00:54<01:22, 27.34s/it]

...

test_loss = 0.321892 , test_acc = 0.879992
train_loss = 0.30, train_acc = 0.892133


 80%|████████  | 4/5 [01:49<00:27, 27.37s/it]

test_loss = 0.305855 , test_acc = 0.888179
train_loss = 0.29, train_acc = 0.896033


100%|██████████| 5/5 [02:20<00:00, 28.18s/it]

test_loss = 0.327661 , test_acc = 0.881989
